In [1]:
import pandas as pd
path = '../데이터/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/'

In [2]:
team_pitcher_2016 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv', encoding='utf-8-sig')
team_pitcher_2017 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv', encoding='utf-8-sig')
team_pitcher_2018 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv', encoding='utf-8-sig')
team_pitcher_2019 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv', encoding='utf-8-sig')
team_pitcher_2020 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2020.csv', encoding='utf-8-sig')
team_pitcher_list = [team_pitcher_2016, team_pitcher_2017, team_pitcher_2018, team_pitcher_2019, team_pitcher_2020]
team_pitcher = pd.concat(team_pitcher_list)

In [ ]:
for i in team_pitcher_list:
    print(i.info())

In [ ]:
team_pitcher

In [3]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP"
new_columns = new_columns.split(',')

In [4]:
team_pitcher_2016.columns = new_columns
team_pitcher_2017.columns = new_columns
team_pitcher_2018.columns = new_columns
team_pitcher_2019.columns = new_columns
team_pitcher_2020.columns = new_columns
team_pitcher.columns = new_columns

In [5]:
team_pitcher = team_pitcher[['게임키', '일자', '팀코드', '상대팀코드','결과', '실책', '실점', '자책점']]
team_pitcher

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점
0,20160401HHLG0,20160401,LG,HH,W,0,4,4
1,20160401HHLG0,20160401,HH,LG,L,0,5,4
2,20160401HTNC0,20160401,NC,HT,W,0,4,4
3,20160401HTNC0,20160401,HT,NC,L,0,5,5
4,20160401KTSK0,20160401,SK,KT,L,0,8,8
...,...,...,...,...,...,...,...,...
635,20200719LTSS0,20200719,LT,SS,W,0,1,1
636,20200719OBHT0,20200719,HT,OB,L,0,8,8
637,20200719OBHT0,20200719,OB,HT,W,0,4,4
638,20200719WOSK0,20200719,SK,WO,W,0,3,3


In [6]:
team_pitcher['일자'] = pd.to_datetime(team_pitcher['일자'],format='%Y%m%d', errors='raise')
team_pitcher['시즌'] = team_pitcher['일자'].dt.year

In [7]:
def gainCreate(row):
    row['득점'] = int(team_pitcher[(row['게임키'] == team_pitcher['게임키']) & (team_pitcher['팀코드'] == row['상대팀코드'])]['실점'])
    return row

team_pitcher = team_pitcher.apply(gainCreate, axis=1)

In [8]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점
0,20160401HHLG0,2016-04-01,LG,HH,W,0,4,4,2016,5
1,20160401HHLG0,2016-04-01,HH,LG,L,0,5,4,2016,4
2,20160401HTNC0,2016-04-01,NC,HT,W,0,4,4,2016,5
3,20160401HTNC0,2016-04-01,HT,NC,L,0,5,5,2016,4
4,20160401KTSK0,2016-04-01,SK,KT,L,0,8,8,2016,4
...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,W,0,1,1,2020,2
636,20200719OBHT0,2020-07-19,HT,OB,L,0,8,8,2020,4
637,20200719OBHT0,2020-07-19,OB,HT,W,0,4,4,2020,8
638,20200719WOSK0,2020-07-19,SK,WO,W,0,3,3,2020,4


In [9]:
def createInfo(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전총득점'] = 0
        row['시작전총실점'] = 0
        row['시작전평균득점'] = 0
        row['시작전평균실점'] = 0
    else:
        gainSum = previous['득점'].sum()
        gainMean = gainSum / len(previous)
        lossSum = previous['실점'].sum()
        lossMean = lossSum / len(previous)
        row['시작전총득점'] = gainSum
        row['시작전총실점'] = lossSum
        row['시작전평균득점'] = gainMean
        row['시작전평균실점'] = lossMean
    return row
team_pitcher = team_pitcher.apply(createInfo, axis=1)

In [10]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점
0,20160401HHLG0,2016-04-01,LG,HH,W,0,4,4,2016,5,0,0,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,L,0,5,4,2016,4,0,0,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,W,0,4,4,2016,5,0,0,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,L,0,5,5,2016,4,0,0,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,L,0,8,8,2016,4,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,W,0,1,1,2020,2,305,320,5.000000,5.245902
636,20200719OBHT0,2020-07-19,HT,OB,L,0,8,8,2020,4,307,282,5.032787,4.622951
637,20200719OBHT0,2020-07-19,OB,HT,W,0,4,4,2020,8,387,348,6.142857,5.523810
638,20200719WOSK0,2020-07-19,SK,WO,W,0,3,3,2020,4,248,338,3.875000,5.281250


In [11]:
def winP(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전승수'] = 0
        row['시작전패배수'] = 0
        row['시작전승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['시작전승수'] = winSum
        row['시작전패배수'] = lossSum
        row['시작전승률'] = winMean
    return row
team_pitcher = team_pitcher.apply(winP, axis=1)

In [17]:
team_pitcher.head(1443)

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률
0,20160401HHLG0,2016-04-01,LG,HH,W,0,4,4,2016,5,0,0,0.000000,0.000000,0,0,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,L,0,5,4,2016,4,0,0,0.000000,0.000000,0,0,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,W,0,4,4,2016,5,0,0,0.000000,0.000000,0,0,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,L,0,5,5,2016,4,0,0,0.000000,0.000000,0,0,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,L,0,8,8,2016,4,0,0,0.000000,0.000000,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,20161009WOLT0,2016-10-09,LT,WO,W,0,5,5,2016,8,769,860,5.377622,6.013986,65,78,0.454545
1439,20161009WOLT0,2016-10-09,WO,LT,L,0,8,4,2016,5,808,749,5.650350,5.237762,77,65,0.538462
0,20170331HHOB0,2017-03-31,OB,HH,W,0,0,0,2017,3,0,0,0.000000,0.000000,0,0,0.000000
1,20170331HHOB0,2017-03-31,HH,OB,L,0,3,1,2017,0,0,0,0.000000,0.000000,0,0,0.000000


In [18]:
def verseWinP(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전상대승수'] = 0
        row['시작전상대패배수'] = 0
        row['시작전상대승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['시작전상대승수'] = winSum
        row['시작전상대패배수'] = lossSum
        row['시작전상대승률'] = winMean
    return row
team_pitcher = team_pitcher.apply(verseWinP, axis=1)

In [20]:
team_pitcher.tail(20)

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률,시작전상대승수,시작전상대패배수,시작전상대승률
620,20200718HHLG0,2020-07-18,LG,HH,W,0,1,1,2020,3,339,325,5.467742,5.241935,32,29,0.516129,7,0,1.000000
621,20200718HHLG0,2020-07-18,HH,LG,L,0,3,2,2020,1,220,377,3.492063,5.984127,17,46,0.269841,0,7,0.000000
622,20200718KTNC0,2020-07-18,NC,KT,W,0,1,1,2020,6,389,292,6.377049,4.786885,40,19,0.655738,5,1,0.714286
623,20200718KTNC0,2020-07-18,KT,NC,L,0,6,6,2020,1,363,343,5.854839,5.532258,31,30,0.500000,1,5,0.142857
624,20200718LTSS0,2020-07-18,SS,LT,W,0,2,2,2020,5,340,311,5.396825,4.936508,33,30,0.523810,5,2,0.714286
625,20200718LTSS0,2020-07-18,LT,SS,L,0,5,5,2020,2,303,315,5.050000,5.250000,29,31,0.483333,2,5,0.285714
626,20200718OBHT0,2020-07-18,HT,OB,L,0,6,6,2020,0,307,276,5.116667,4.600000,33,27,0.550000,2,5,0.285714
627,20200718OBHT0,2020-07-18,OB,HT,W,0,0,0,2020,6,381,348,6.145161,5.612903,36,26,0.580645,5,2,0.714286
628,20200718WOSK0,2020-07-18,SK,WO,L,0,4,4,2020,1,247,334,3.920635,5.301587,20,43,0.317460,2,5,0.285714
629,20200718WOSK0,2020-07-18,WO,SK,W,0,1,1,2020,4,363,331,5.671875,5.171875,37,27,0.578125,5,2,0.714286


In [23]:
team_pitcher.to_csv("../EDA결과/EDA2-경기전 팀 스탯.csv", encoding='utf-8-sig')

In [ ]:
t